In [5]:
## import pandas and numpy libraries to work with dataframes and perform calculations
import pandas as pd
import numpy as np

In [6]:
## import sklearn and relevant libaries for machine learning
import sklearn
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV as CCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

In [7]:
fav = pd.read_csv('favorite_underdog_data(1).csv')
fav = fav.loc[:, 'schedule_date'::]
# change data type of date columns
fav['schedule_date'] = fav['schedule_date'].astype(str)

In [9]:
fav.columns

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'favorite', 'underdog', 'score_favorite', 'score_underdog',
       'spread_favorite', 'over_under_line', 'stadium', 'stadium_neutral',
       'home_favorite', 'away_favorite', 'over', 'elo_pre_favorite',
       'elo_pre_underdog', 'elo_prob_favorite', 'elo_prob_underdog',
       'qb_favorite', 'qb_underdog', 'qb_value_pre_favorite',
       'qb_value_pre_underdog', 'qb_adj_favorite', 'qb_adj_underdog',
       'qbelo_prob_favorite', 'qbelo_prob_underdog', 'result',
       'favorite_cover', 'elo_pre_diff', 'elo_prob_diff', 'qb_value_pre_diff',
       'qb_adj_diff', 'qbelo_prob_diff', 'margin',
       'favorite_predictive_rating', 'underdog_predictive_rating',
       'predictive_rating_difference', 'favorite_predictive_rank',
       'underdog_predictive_rank', 'predictive_rank_difference',
       'favorite_home_rating', 'underdog_home_rating',
       'home_rating_difference', 'favorite_home_rank', 'under

In [75]:
features = fav[['schedule_season',
       'home_favorite', 'away_favorite', 'elo_pre_favorite',
       'elo_pre_underdog', 'elo_prob_favorite', 'elo_prob_underdog', 'qb_value_pre_favorite',
       'qb_value_pre_underdog', 'qb_adj_favorite', 'qb_adj_underdog',
       'qbelo_prob_favorite', 'qbelo_prob_underdog', 'elo_pre_diff', 'elo_prob_diff', 'qb_value_pre_diff',
       'qb_adj_diff', 'qbelo_prob_diff',
       'favorite_predictive_rating', 'underdog_predictive_rating',
       'predictive_rating_difference', 'favorite_predictive_rank',
       'underdog_predictive_rank', 'predictive_rank_difference',
       'favorite_home_rating', 'underdog_home_rating',
       'home_rating_difference', 'favorite_home_rank', 'underdog_home_rank',
       'home_rank_difference', 'favorite_away_rating', 'underdog_away_rating',
       'away_rating_difference', 'favorite_away_rank', 'underdog_away_rank',
       'away_rank_difference', 'favorite_location_rating',
       'underdog_location_rating', 'location_difference_rating',
       'favorite_location_rank', 'underdog_location_rank',
       'location_difference_rank', 'favorite_ppg', 'underdog_ppg',
       'favorite_ppg_recent', 'underdog_ppg_recent', 'favorite_ppg_rank',
       'underdog_ppg_rank', 'ppg_diff', 'ppg_recent_diff', 'ppg_rank_diff',
       'favorite_margin', 'underdog_margin', 'favorite_margin_recent',
       'underdog_margin_recent', 'favorite_margin_rank',
       'underdog_margin_rank', 'margin_diff', 'margin_recent_diff',
       'margin_rank_diff', 'favorite_redzone', 'underdog_redzone',
       'favorite_redzone_recent', 'underdog_redzone_recent',
       'favorite_redzone_rank', 'underdog_redzone_rank', 'redzone_diff',
       'redzone_recent_diff', 'redzone_rank_diff', 'favorite_ypp',
       'underdog_ypp', 'favorite_ypp_recent', 'underdog_ypp_recent',
       'favorite_ypp_rank', 'underdog_ypp_rank', 'ypp_diff', 'ypp_recent_diff',
       'ypp_rank_diff']]

## features with correlation to favorite score of >0.14 or <-0.14
best_score1_features = fav[["elo_prob_favorite", "qbelo_prob_favorite","favorite_ypp","favorite_ppg","qb_value_pre_favorite","elo_prob_diff",
                   "qbelo_prob_diff", "favorite_predictive_rating", "favorite_ppg_recent", "elo_prob_underdog", "qbelo_prob_underdog",
                   "favorite_ppg_rank", "favorite_predictive_rank"]]

## features with correlation to underdog score of >0.14 or <-0.14
best_score2_features = fav[["elo_prob_favorite", "elo_prob_underdog", "qb_value_pre_underdog", "qbelo_prob_favorite", "qbelo_prob_underdog",
                           "elo_pre_diff", "elo_prob_diff", "qbelo_prob_diff", "underdog_ppg", "underdog_ppg_rank", 
                           "underdog_ypp", "underdog_ypp_recent"]]

score_1 = fav["score_favorite"]
score_2 = fav["score_underdog"]

In [76]:
fav.corr()

,schedule_season,schedule_week,schedule_playoff,score_favorite,score_underdog,spread_favorite,over_under_line,stadium_neutral,home_favorite,away_favorite,over,elo_pre_favorite,elo_pre_underdog,elo_prob_favorite,elo_prob_underdog,qb_value_pre_favorite,qb_value_pre_underdog,qb_adj_favorite,qb_adj_underdog,qbelo_prob_favorite,qbelo_prob_underdog,result,favorite_cover,elo_pre_diff,elo_prob_diff,qb_value_pre_diff,qb_adj_diff,qbelo_prob_diff,margin,favorite_predictive_rating,underdog_predictive_rating,predictive_rating_difference,favorite_predictive_rank,underdog_predictive_rank,predictive_rank_difference,favorite_home_rating,underdog_home_rating,home_rating_difference,favorite_home_rank,underdog_home_rank,home_rank_difference,favorite_away_rating,underdog_away_rating,away_rating_difference,favorite_away_rank,underdog_away_rank,away_rank_difference,favorite_location_rating,underdog_location_rating,location_difference_rating,favorite_location_rank,underdog_location_rank,location_difference_rank,favorite_ppg,underdog_ppg,favorite_ppg_recent,underdog_ppg_recent,favorite_ppg_rank,underdog_ppg_rank,ppg_diff,ppg_recent_diff,ppg_rank_diff,favorite_margin,underdog_margin,favorite_margin_recent,underdog_margin_recent,favorite_margin_rank,underdog_margin_rank,margin_diff,margin_recent_diff,margin_rank_diff,favorite_redzone,underdog_redzone,favorite_redzone_recent,underdog_redzone_recent,favorite_redzone_rank,underdog_redzone_rank,redzone_diff,redzone_recent_diff,redzone_rank_diff,favorite_ypp,underdog_ypp,favorite_ypp_recent,underdog_ypp_recent,favorite_ypp_rank,underdog_ypp_rank,ypp_diff,ypp_recent_diff,ypp_rank_diff
schedule_season,1.000000,-0.054023,-0.011309,0.055187,0.096269,-0.028760,0.380142,0.064228,-0.030967,0.038742,-0.006455,0.012052,0.010444,-0.023785,0.023785,0.401334,0.391201,0.033708,-0.050453,0.005363,-0.005363,-0.019297,-0.019591,0.001148,-0.023785,0.042214,0.061207,0.005363,-0.026542,0.088194,-0.085491,0.147577,-0.023034,0.031961,-0.046085,-0.049520,-0.064943,0.015243,-0.014515,0.022013,-0.028439,0.023386,0.071167,-0.037511,0.018334,-0.008024,0.020303,-0.026472,0.017477,-0.032886,0.003073,0.010820,-0.006268,0.177902,0.181742,0.132429,0.118487,0.000601,0.007511,0.005250,0.015267,-0.005375,-0.006987,-0.001502,0.003806,-0.009248,0.003877,0.007681,-0.004157,0.010123,-0.003275,0.168044,0.158137,0.118688,0.115624,0.008224,-0.005797,0.003472,-0.002674,0.009953,0.218828,0.238477,0.177817,0.178301,0.006604,-0.010096,-0.017540,-0.001054,0.012277
schedule_week,-0.054023,1.000000,0.421650,0.011638,-0.027217,0.068812,0.007681,0.040638,0.008434,-0.009422,-0.028379,0.163596,0.079703,0.070901,-0.070901,0.043684,-0.005485,0.019136,-0.042037,0.097458,-0.097458,0.026937,0.013865,0.070636,0.070901,0.042293,0.045349,0.097458,0.027426,0.158171,0.060470,0.078226,-0.094095,-0.096657,0.005204,0.059790,0.054324,0.000781,-0.119966,-0.046240,-0.054709,0.060561,0.073062,-0.010673,-0.119798,-0.046234,-0.055733,0.075091,0.047812,0.018430,-0.128980,-0.037718,-0.070530,0.075713,0.088240,0.060103,0.064093,-0.123267,-0.031574,-0.006059,-0.000749,-0.069517,0.050418,0.085070,0.059086,0.078130,-0.141718,-0.048467,-0.030080,-0.016230,-0.073444,0.014592,0.051090,0.032008,0.046178,-0.060298,-0.014306,-0.027567,-0.011830,-0.032421,0.016852,0.043122,-0.008236,0.006391,-0.086915,-0.015165,-0.020370,-0.010915,-0.052072
schedule_playoff,-0.011309,0.421650,1.000000,0.012573,0.027381,0.009999,0.060513,0.098320,0.084251,-0.081299,-0.000848,0.239274,0.317917,-0.022594,0.022594,0.115197,0.137520,0.058202,0.053830,0.028092,-0.028092,-0.009262,-0.009459,-0.075284,-0.022594,-0.007671,-0.005873,0.028092,-0.009834,0.221024,0.277985,-0.059424,-0.173342,-0.270503,0.088295,0.122458,0.179456,-0.052715,-0.166044,-0.226196,0.052467,0.104814,0.181346,-0.061146,-0.143129,-0.229466,0.069263,0.111119,0.182799,-0.058847,-0.164806,-0.223770,0.050536,0.106809,0.160786,0.076267,0.162941,-0.149398,-0.206641,-0.036189,-0.061684,0.046522,0.126712,0.197766,0.113348,0.209620,-0.183102,-0.256863,-0.

In [82]:
## import and intialize scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

## fit scaler to data
scaler.fit(best_score2_features)
features_scaled = scaler.transform(best_score2_features)

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


estimators = {
    'Linear Regression': LinearRegression(), 
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'K-Nearest-Neighbors': KNeighborsRegressor(n_neighbors = 3)}

In [85]:
def classifiers_percentage_split():
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, score_2, random_state=37)
    for estimator_name, estimator_object in estimators.items():
        estimator_object.fit(X=X_train, y=y_train)
        accuracy = estimator_object.score(X_test, y_test)
        print(estimator_name)
        print("\t Prediction accuracy on the test data:", f"{accuracy:.2%} \n")

def classifiers_cross_validation():
    for estimator_name, estimator_object in estimators.items():
        kfold = KFold(n_splits=10, random_state=37, shuffle=True)
        scores = cross_val_score(estimator_object, X=features, y=score_2, cv=kfold)
        print(estimator_name + ": \n\t" + f'mean accuracy={scores.mean():.2%}, ' + f'standard deviation={scores.std():.2%}' +"\n")

In [86]:
classifiers_percentage_split()

Linear Regression
	 Prediction accuracy on the test data: 7.05% 

Ridge Regression
	 Prediction accuracy on the test data: 7.06% 

Lasso Regression
	 Prediction accuracy on the test data: 6.30% 

K-Nearest-Neighbors
	 Prediction accuracy on the test data: -24.79% 



In [87]:
classifiers_cross_validation()

Linear Regression: 
	mean accuracy=6.60%, standard deviation=3.69%

Ridge Regression: 
	mean accuracy=6.74%, standard deviation=3.50%

Lasso Regression: 
	mean accuracy=7.09%, standard deviation=3.14%

K-Nearest-Neighbors: 
	mean accuracy=-24.31%, standard deviation=3.44%



In [88]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target)

def RFE_feature_selection():

    select = RFE(DecisionTreeRegressor(), n_features_to_select=10)
    
    #fit the RFE selector to the training data
    select.fit(X_train, y_train)

    #transform training and testing sets so only the selected features are retained
    X_train_selected = select.transform(X_train)
    X_test_selected = select.transform(X_test)
    
    model = DecisionTreeClassifier(max_depth = 40).fit(X=X_train_selected, y=y_train)
    
    print("Selected features after RFE:")
    for i in range(len(select.get_support())):
        if select.get_support()[i]:
            print(features.columns[i]) #gets the features for which get_support is true
    
    accuracy_train = model.score(X_train_selected, y_train)
    accuracy_test = model.score(X_test_selected, y_test)

    print("\nkNN Classification performance with selected features:")
    print("\t Prediction accuracy on the train data:", f"{accuracy_train:.2%} \n")
    print("\t Prediction accuracy on the test data:", f"{accuracy_test:.2%} \n")

RFE_feature_selection()

Selected features after RFE:
home_favorite
away_favorite
elo_pre_favorite
elo_pre_underdog
elo_prob_favorite
qb_value_pre_favorite
qb_value_pre_underdog
qb_adj_favorite
qb_adj_underdog
qbelo_prob_favorite

kNN Classification performance with selected features:
	 Prediction accuracy on the train data: 100.00% 

	 Prediction accuracy on the test data: 4.62% 



In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target)

def RFE_feature_selection():

    select = RFE(DecisionTreeRegressor(), n_features_to_select=9)
    
    #fit the RFE selector to the training data
    select.fit(X_train, y_train)

    #transform training and testing sets so only the selected features are retained
    X_train_selected = select.transform(X_train)
    X_test_selected = select.transform(X_test)
    
    model = Ridge().fit(X=X_train_selected, y=y_train)
    
    print("Selected features after RFE:")
    for i in range(len(select.get_support())):
        if select.get_support()[i]:
            print(features.columns[i]) #gets the features for which get_support is true
    
    accuracy_train = r2_score(y_train, model.predict(X_train_selected))
    accuracy_test = r2_score(y_test, model.predict(X_test_selected))

    print("\nkNN Classification performance with selected features:")
    print("\t Prediction accuracy on the train data:", f"{accuracy_train} \n")
    print("\t Prediction accuracy on the test data:", f"{accuracy_test} \n")

RFE_feature_selection()